# HORNET Attack Demo Notebook

This notebook demonstrates how to run the HORNET attack on CIFAR-10 and evaluate its transferability.

Ensure you are in the project root directory (`HORNET/`) for relative imports to work.

## 1. Environment Setup & Imports

In [ ]:
# First, let's install required packages if not already present in your environment.
# Uncomment and run the following cell if needed:
!conda env create -f environment.yml
!conda activate attackbench-HORNET

import sys
import os
import subprocess
import json
import pandas as pd
import matplotlib.pyplot as plt

# Add project root to Python path to import local modules
sys.path.append('..')

# Verify essential directories exist
assert os.path.exists('attack_evaluation'), "Directory 'attack_evaluation/' not found. Ensure you are in the correct working directory (HORNET/)."

## 2. Define Attack Configuration

In [ ]:
# We'll run HORNET using the project's command-line interface.
# This cell defines the configuration for a single attack run.

attack_config = {
    "attack": "art_fgm_hornet",
    "source_model": "standard",
    "target_models": ["zhang_2020_small", "stutz_2020", "xiao_2020", "wang_2023_small"],
    "num_samples": 100,
    "batch_size": 32,
    "threat_model": "l2",
    "seed": 42
}

# Let's create a descriptive name for the results folder based on the configuration.
results_folder_name = "results_demo"
print(f"Results will be saved to: {results_folder_name}")

## 3. Execute the Attack

In [ ]:
# We run the attack via a subprocess call, matching the project's standard usage.
print("Running HORNET attack. This may take several minutes...")
# Construct the command
cmd = [
    "python", "-m", "attack_evaluation.run_with_trans",
    "-F", results_folder_name,
    "with",
    f"attack.{attack_config['attack']}",
    f"model.{attack_config['source_model']}",
    f"model.target_models={json.dumps(attack_config['target_models'])}",
    f"dataset.num_samples={attack_config['num_samples']}",
    f"dataset.batch_size={attack_config['batch_size']}",
    f"attack.threat_model={attack_config['threat_model']}",
    f"seed={attack_config['seed']}"
]

# Run the command and capture output
try:
    result = subprocess.run(cmd, capture_output=True, text=True, check=True)
    print("Attack execution completed successfully.")
    print("STDOUT:", result.stdout[:500])  # Print first 500 chars of output
except subprocess.CalledProcessError as e:
    print("Error during attack execution:")
    print("STDERR:", e.stderr)
    raise e

## 4. Analysis results

In [ ]:
analysis_cmd = [
    "python", "-m", "analysis.print_tables_and_plot",
    "-d", results_folder_name,
    "-dist", "actual",
    "--source_models", attack_config["source_model"],
    "--target_models"
] + attack_config["target_models"]

try:
    analysis_result = subprocess.run(analysis_cmd, capture_output=True, text=True)
    print("Analysis script executed.")
    # The script typically prints summary to stdout. Let's capture key lines.
    for line in analysis_result.stdout.split('\n'):
        if line.strip() and ('ASR' in line or 'success' in line.lower() or 'average' in line.lower()):
            print(line.strip())
except Exception as e:
    print(f"Note: Standard analysis script encountered an issue: {e}")
    print("We will attempt to load results directly from the generated files.")